In [495]:
#import libraries

import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import hvplot.pandas
import hvplot
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
import warnings

# ignore any non-critical future warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [496]:
#load .env file

load_dotenv()

True

In [497]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [498]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [499]:
# Request stock symbol from user. This code will work will all available tickers on alpacas.
# "META" ticker has been temporarily hard coded into the system for ease of use

# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")

stock_symbol = "QQQ"

In [500]:
# Establish what time frame increment we will be looking at
# currently we are working with the daily chart, but the theory would be to 
# allow the user to identify what time scale they would like to trade on
# shorter time frame allows for "day trades" larger time scales are geared
# more towards longer term investments and swing trading.

time_frame = "1Day"

In [501]:
# Get user input for the start date in datetime format
# the start date has been hard coded for ease of programming and troubleshooting our code

# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2016-01-01", tz="America/New_York").isoformat()

In [502]:
# Get user input for the end date in datetime format
# the end date has been hard coded for ease of programming and troubleshooting our code

# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

In [503]:
# trading_days = end_date - start_date

# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [504]:
# Calls the stock data from the period of time desired in the desired time increment

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [505]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2016-01-04 05:00:00+00:00 to 2019-12-31 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        1006 non-null   float64
 1   high         1006 non-null   float64
 2   low          1006 non-null   float64
 3   trade_count  1006 non-null   int64  
 4   open         1006 non-null   float64
 5   volume       1006 non-null   int64  
 6   vwap         1006 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 62.9 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2016-01-04 05:00:00+00:00,109.50,109.60,108.12,181682,109.43,51517062,109.242476
2016-01-05 05:00:00+00:00,109.31,110.18,108.80,151825,110.04,39065561,109.446365
2016-01-06 05:00:00+00:00,108.26,108.89,107.41,156916,107.43,42234893,108.171269
2016-01-07 05:00:00+00:00,104.87,107.29,104.81,241719,105.96,61742257,105.987995
2016-01-08 05:00:00+00:00,104.01,106.29,103.90,239124,105.67,69707476,105.115694


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2019-12-24 05:00:00+00:00,211.92,212.09,211.44,31461,211.97,7239625,211.797784
2019-12-26 05:00:00+00:00,213.79,213.81,212.23,64174,212.26,17568713,213.142782
2019-12-27 05:00:00+00:00,213.61,214.56,213.04,74578,214.55,18557414,213.779548
2019-12-30 05:00:00+00:00,212.21,213.63,211.16,95582,213.51,22179426,212.229443
2019-12-31 05:00:00+00:00,212.61,212.76,211.20,83257,211.62,18503690,212.114620


In [506]:
# Reset index and display the first 5
# this also prepares for use with plotly library/visualization

#stock_data_plot = stock_data.reset_index()

# clean up datafram by removing vwap, volume, trade count columns
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results

stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
stock_data_plot = stock_data.reset_index()

stock_data.head()

,close,high,low,open
timestamp,,,,
2016-01-04 05:00:00+00:00,109.50,109.60,108.12,109.43
2016-01-05 05:00:00+00:00,109.31,110.18,108.80,110.04
2016-01-06 05:00:00+00:00,108.26,108.89,107.41,107.43
2016-01-07 05:00:00+00:00,104.87,107.29,104.81,105.96
2016-01-08 05:00:00+00:00,104.01,106.29,103.90,105.67


In [507]:
# Assign plot values to plotly

fig = go.Candlestick(x=stock_data_plot['timestamp'],
                open=stock_data_plot['open'],
                high=stock_data_plot['high'],
                low=stock_data_plot['low'],
                close=stock_data_plot['close'])


cand = go.Figure(data=[fig])

In [508]:
# Customize the figure (optional)

cand.update_layout(
    width=1200, height=800,
    title="AAPL, 2016 - 2020",
    yaxis_title='AAPL Stock Price'
)


In [509]:
# Add new data columns to DF for body size and candle size
# body size and direction is value of open - close. 
# candle size is the absolute value of low - high

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

# Display columns with added columns

stock_data.head()

,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2016-01-04 05:00:00+00:00,109.50,109.60,108.12,109.43,0.07,1.48
2016-01-05 05:00:00+00:00,109.31,110.18,108.80,110.04,-0.73,1.38
2016-01-06 05:00:00+00:00,108.26,108.89,107.41,107.43,0.83,1.48
2016-01-07 05:00:00+00:00,104.87,107.29,104.81,105.96,-1.09,2.48
2016-01-08 05:00:00+00:00,104.01,106.29,103.90,105.67,-1.66,2.39


In [510]:
# initialize variables to be used in calculations
# identify the max body size and minimum candle size within the dataframe

max_body_size = stock_data["body_size"].max()
min_candle_size = stock_data["candle_size"].min()

# keeps track of the count which would initiate a buy or sell
# current count is initialized at 0 as the count is calculated based off of historical data

current_count = 0

# the threshold that would initiate a buy and or sell

threshold = 3

# adds a documenting column to the dataframe and assigns a flat value to all rows

stock_data["card_count"] = current_count

# identifies the number of rows to be used to break the for loop

number_of_rows = len(stock_data)

# print information up to date to identify data progress

# print(max_body_size)
# print(min_candle_size)
# print(previous_candle)
# print(stock_data.iloc[0][4])
# print(number_of_rows)
# stock_data.head()

In [511]:
# ...::: working code :::...
# initialize variables and dataframe for tracking buys, sells, and profits

initial_investment = 10000

# initialize strategy tracking variables and add to stock_data df

stock_data["capital"] = initial_investment
stock_data["number_of_shares"] = 0
stock_data["investment"] = 0
stock_data["profits"] = 0

# variables that will be used to calculate profits/losses from buys/sells

capital = initial_investment
number_of_shares = 0
investment = 0
profits = 0

stock_data.head()

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2016-01-04 05:00:00+00:00,109.50,109.60,108.12,109.43,0.07,1.48,0,10000,0,0,0
2016-01-05 05:00:00+00:00,109.31,110.18,108.80,110.04,-0.73,1.38,0,10000,0,0,0
2016-01-06 05:00:00+00:00,108.26,108.89,107.41,107.43,0.83,1.48,0,10000,0,0,0
2016-01-07 05:00:00+00:00,104.87,107.29,104.81,105.96,-1.09,2.48,0,10000,0,0,0
2016-01-08 05:00:00+00:00,104.01,106.29,103.90,105.67,-1.66,2.39,0,10000,0,0,0


In [512]:
# initialize variables and assign values based off of first row for calculations

previous_close = stock_data.iloc[0][0]
previous_high = stock_data.iloc[0][1]
previous_low = stock_data.iloc[0][2]
previous_open = stock_data.iloc[0][3]
previous_body_size = stock_data.iloc[0][4]
previous_candle_size = stock_data.iloc[0][5]

# begin for loop to start calculating each row's data to assign a +1 or -1
# to our count and add adjust our current/previous counts accordingly

for index, row in islice(stock_data.iterrows(), 1, None):

# create an if statement to make sure the for loop finishes at the end of the data appropriately  
    
    if index == (number_of_rows):
        break
    
# assign current day numbers to work with conditional statements

    current_close = row[0]
    current_high = row[1]
    current_low = row[2]
    current_open = row[3]
    current_body_size = row[4]
    current_candle_size = row[5]
    # capital = row[7]
    # number_of_shares = row[8]
    # investment = row[9]
    # profits = row[10]

   # begin of conditional statements. this portion identifies what values to assign
   # to each individal row's data and to determine whether a threshold of "sell" or "buy"
   # is reached. this portion can be adjusted/added to in order to take into consideration
   # additional bar patterns

    # bullish hammer
    if (current_body_size > 0) and (current_body_size*2) < (current_candle_size):
        current_count = current_count + 1
    # bearish hammer
    elif (current_body_size < 0) and ((np.absolute(current_body_size)*2) < (current_candle_size)):
        current_count = current_count - 1
    # bullish engulfing candle
    elif (current_body_size > 0) and (previous_body_size < 1) and (current_body_size > previous_body_size):
        current_count = current_count + 1
    # bearish engulfing candle
    elif (current_body_size < 0) and (previous_body_size > 1) and (np.absolute(current_body_size) > (previous_body_size)):
        current_count = current_count - 1
    # bullish doji candle
    #elif (previous_body_size < 0) and (np.absolute(current_body_size) < (current_close * max_body_size)):
    #    current_count = current_count + 1

    # contains the current count actual value to the desired "buy/sell" threshold

    if current_count > threshold:
        current_count = threshold
    elif current_count < -(threshold):
        current_count = -(threshold)

    # conditional statements to identify when to purchase or sell a stock

    if (current_count == threshold) and (number_of_shares == 0):
        number_of_shares = np.floor(capital / current_close)
        investment = number_of_shares * current_close
        capital = capital - investment
    elif (current_count == -threshold) and (number_of_shares != 0):
        profits = (number_of_shares * current_close) - investment
        capital = capital + investment + profits
        investment = 0
        number_of_shares = 0
    else:
        profits = (number_of_shares * current_close) - investment

        #stock_data.at[index,"investment"] = number_of_shares * current_close
        #profits = row[9] - investment
        #stock_data.at[index,"profits"] = profits

    stock_data.at[index,"number_of_shares"] = number_of_shares
    stock_data.at[index,"investment"] = investment
    stock_data.at[index,"profits"] = profits
    stock_data.at[index,"capital"] = capital
    # set variables to prepare to analyze the following row

    previous_close = row[0]
    previous_high = row[1]
    previous_low = row[2]
    previous_open = row[3]
    previous_body_size = row[4]
    previous_candle_size = row[5]

    # keep track of the current count within the dataframe. assign that
    # to the current working row
    
    stock_data.at[index,"card_count"] = current_count

if (number_of_shares == 0):
    total_profit = capital
elif (number_of_shares != 0):
    total_profit = (capital + investment + profits) - initial_investment

total_profit = round(total_profit,2)
percent_profit = (total_profit/initial_investment)*100

# prints total profits and percentage profits over analyze time period

print(f"total profits for your autotrader are: ${total_profit} a {percent_profit}%")

# display dataframe for processing checks

#display(stock_data.head(40))
#display(stock_data.tail(40))





total profits for your autotrader are: $10481.83 a 104.81830000000001%


In [513]:
# comparative analysis calculations
# comparing buying and holding vs card counting algorithm

comp_number_of_shares = np.floor(initial_investment / stock_data.iloc[0][0])
stock_data["comp_profits"] = (comp_number_of_shares * stock_data["close"]) - initial_investment
stock_data["total_profits"] = (stock_data["capital"] + stock_data["investment"] + stock_data["profits"]) - initial_investment
# print(comp_number_of_shares)
# display(stock_data.head(40))
# display(stock_data.tail(40))


In [514]:
# Plot the Comparison between Total Profits and Comparative Profits

(stock_data.hvplot.line(
    x="timestamp",
    y=('total_profits'),
    title= 'Buy & Hold vs. Card Counting Algorithm (QQQ)',
    label = ("Card Counting")
    )*stock_data.hvplot.line(
    y="comp_profits",
    label = "Buy & Hold")).opts(legend_position = "top_left")


:Overlay
   .Curve.Buy & Hold         :Curve   [timestamp]   (total_profits)
   .Curve.Buy_ampersand_Hold :Curve   [timestamp]   (comp_profits)

In [515]:
# Plot the Number of Shares in Line Graph

stock_data.hvplot.line(x='timestamp', y='number_of_shares', title='Shares Held Throughout Time (AAPL)')

:Curve   [timestamp]   (number_of_shares)

In [516]:
# Analysis results dataframe and display dataframe

stock_data['port_size'] = stock_data['capital'] + stock_data['profits'] + stock_data['investment']

display(stock_data.head())
display(stock_data.tail())

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,109.50,109.60,108.12,109.43,0.07,1.48,0,10000.0,0,0.0,0.0,-35.50,0.0,10000.0
2016-01-05 05:00:00+00:00,109.31,110.18,108.80,110.04,-0.73,1.38,0,10000.0,0,0.0,0.0,-52.79,0.0,10000.0
2016-01-06 05:00:00+00:00,108.26,108.89,107.41,107.43,0.83,1.48,1,10000.0,0,0.0,0.0,-148.34,0.0,10000.0
2016-01-07 05:00:00+00:00,104.87,107.29,104.81,105.96,-1.09,2.48,0,10000.0,0,0.0,0.0,-456.83,0.0,10000.0
2016-01-08 05:00:00+00:00,104.01,106.29,103.90,105.67,-1.66,2.39,0,10000.0,0,0.0,0.0,-535.09,0.0,10000.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2019-12-24 05:00:00+00:00,211.92,212.09,211.44,211.97,-0.05,0.65,-1,71.27,96,15854.4,4489.92,9284.72,10415.59,20415.59
2019-12-26 05:00:00+00:00,213.79,213.81,212.23,212.26,1.53,1.58,0,71.27,96,15854.4,4669.44,9454.89,10595.11,20595.11
2019-12-27 05:00:00+00:00,213.61,214.56,213.04,214.55,-0.94,1.52,0,71.27,96,15854.4,4652.16,9438.51,10577.83,20577.83
2019-12-30 05:00:00+00:00,212.21,213.63,211.16,213.51,-1.30,2.47,0,71.27,96,15854.4,4517.76,9311.11,10443.43,20443.43
2019-12-31 05:00:00+00:00,212.61,212.76,211.20,211.62,0.99,1.56,1,71.27,96,15854.4,4556.16,9347.51,10481.83,20481.83


In [517]:
#Plot Line Graph for Portfolio Size

stock_data.hvplot.line(x='timestamp', y='port_size', title='Portfolio Size Over Time (AAPL)')

:Curve   [timestamp]   (port_size)

In [518]:
# Plot a Line Graph for Card Count

stock_data.hvplot.line(x='timestamp', y='card_count', title='Card Count Over Time (AAPL)')

:Curve   [timestamp]   (card_count)

In [519]:
# Plot a Scatter Graph for Card Count

stock_data.hvplot.scatter(x='timestamp', y='card_count', height=400, width=800, title= 'Card Count Over Time (AAPL)')

:Scatter   [timestamp]   (card_count)